--- Day 18: Lavaduct Lagoon ---

Thanks to your efforts, the machine parts factory is one of the first factories up and running since the lavafall came back. However, to catch up with the large backlog of parts requests, the factory will also need a large supply of lava for a while; the Elves have already started creating a large lagoon nearby for this purpose.

However, they aren't sure the lagoon will be big enough; they've asked you to take a look at the dig plan (your puzzle input). For example:

```
R 6 (#70c710)
D 5 (#0dc571)
L 2 (#5713f0)
D 2 (#d2c081)
R 2 (#59c680)
D 2 (#411b91)
L 5 (#8ceee2)
U 2 (#caa173)
L 1 (#1b58a2)
U 2 (#caa171)
R 2 (#7807d2)
U 3 (#a77fa3)
L 2 (#015232)
U 2 (#7a21e3)
```

The digger starts in a 1 meter cube hole in the ground. They then dig the specified number of meters up (U), down (D), left (L), or right (R), clearing full 1 meter cubes as they go. The directions are given as seen from above, so if "up" were north, then "right" would be east, and so on. Each trench is also listed with the color that the edge of the trench should be painted as an RGB hexadecimal color code.

When viewed from above, the above example dig plan would result in the following loop of trench (#) having been dug out from otherwise ground-level terrain (.):

```
#######
#.....#
###...#
..#...#
..#...#
###.###
#...#..
##..###
.#....#
.######
```

At this point, the trench could contain 38 cubic meters of lava. However, this is just the edge of the lagoon; the next step is to dig out the interior so that it is one meter deep as well:

```
#######
#######
#######
..#####
..#####
#######
#####..
#######
.######
.######
```

Now, the lagoon can contain a much more respectable 62 cubic meters of lava. While the interior is dug out, the edges are also painted according to the color codes in the dig plan.

The Elves are concerned the lagoon won't be large enough; if they follow their dig plan, how many cubic meters of lava could it hold?

Your puzzle answer was 50746.

In [ ]:
from collections import deque
import copy
import re

directions = {
    'R': (1, 0),
    'D': (0, 1),
    'L': (-1, 0),
    'U': (0, -1),
}

def read_input(filePath):
    with open(filePath, 'r') as file:
        return file.read()
    
def read_digplan(text):
    matches = re.findall(r'(\w)\s(\d+)\s\(#(.{6})\)', text)
    return [(directions(dir), int(dist)) for (dir, dist, _) in matches]

def print_terrain(terrain):
    xs, ys = zip(*terrain)
    for y in range(min(ys), max(ys) + 1):
        for x in range(min(xs), max(xs) + 1):
            if (x,y) in terrain:
                print('#', end='')
            else :
                print('.', end='')
        print()

def dig_trench(terrain, digplan, start):
    (x, y) = start
    for (dir, dist) in digplan:
        (sx, sy) = dir
        for _ in range(dist):
            x = x + sx
            y = y + sy
            terrain.add((x,y))
    return terrain

def flood_fill(terrain, start):
    stack = deque([start])
    while stack:
        (x, y) = stack.pop()
        if (x,y) not in terrain:
            terrain.add((x,y))
            stack.extend([(x+1, y), (x-1, y), (x, y+1), (x, y-1)])
    return terrain

input = read_input('sample.txt')
digplan = read_digplan(input)

trench = dig_trench(set(), digplan, (0, 0))
filled = flood_fill(set(trench), (1, 1))

print('----------------')
print_terrain(trench)
print('----------------')
print_terrain(filled)
print('----------------')
print(len(filled))

--- Part Two ---

The Elves were right to be concerned; the planned lagoon would be much too small.

After a few minutes, someone realizes what happened; someone swapped the color and instruction parameters when producing the dig plan. They don't have time to fix the bug; one of them asks if you can extract the correct instructions from the hexadecimal codes.

Each hexadecimal code is six hexadecimal digits long. The first five hexadecimal digits encode the distance in meters as a five-digit hexadecimal number. The last hexadecimal digit encodes the direction to dig: 0 means R, 1 means D, 2 means L, and 3 means U.

So, in the above example, the hexadecimal codes can be converted into the true instructions:

```
#70c710 = R 461937
#0dc571 = D 56407
#5713f0 = R 356671
#d2c081 = D 863240
#59c680 = R 367720
#411b91 = D 266681
#8ceee2 = L 577262
#caa173 = U 829975
#1b58a2 = L 112010
#caa171 = D 829975
#7807d2 = L 491645
#a77fa3 = U 686074
#015232 = L 5411
#7a21e3 = U 500254
```

Digging out this loop and its interior produces a lagoon that can hold an impressive 952408144115 cubic meters of lava.

Convert the hexadecimal color codes into the correct instructions; if the Elves follow this new dig plan, how many cubic meters of lava could the lagoon hold?

Your puzzle answer was 70086216556038.

In [ ]:
directions2 = {
    '0': (1, 0),
    '1': (0, 1),
    '2': (-1, 0),
    '3': (0, -1),
}

def read_digplan2(text):
    matches = re.findall(r'(\w)\s(\d+)\s\(#(.{6})\)', text)
    return [(directions2[hex[5]], int(hex[0:5], 16)) for (_, _, hex) in matches]

def dig_trench2(terrain, digplan, start):
    (x, y) = start
    for (dir, dist) in digplan:
        sx, sy = dir
        dx, dy = (sx * dist, sy * dist)
        terrain.append((x, y, dx, dy))
        x, y = (x + dx, y + dy)
    return terrain

#
#   o---------------------> x
#   |    
#   |    .------------>
#   |    |            |
#   |    <---.        |
#   |        |        |
#   |        <--------'
#   v
#
#   Calculating the area of the polygon above is equivalent of adding
#   or subtracting areas between the x-axis and the horizontal edges of
#   the polygon.
#
#   Subtract the area for positive deltas (right) and add the area for
#   negative deltas (left).
#
#   This is the area for a idealized (no border) polygon. Half of the
#   border area is added to the result to compensate.
#
#   This generalizes to all polygons built from rectangles.
#
def calc_fill2(trench):
    box_area = 0
    for (_, y, dx, _) in trench:
        box_area -= y * dx
    edge_area = 0.5 * sum(max(abs(dx), abs(dy)) for (_, _, dx, dy) in trench)
    return box_area + edge_area + 1

input = read_input('sample.txt')
digplan = read_digplan2(input)

trench = dig_trench2([], digplan, (0, 0))
fill = calc_fill2(trench)

print('----------------')
print(trench)
print('----------------')
print(fill)
print('----------------')